In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
floatX = np.float64

In [4]:
mean_truth = np.array([
    [0.2, 0.4],
    [0.5, 0.5],
    [0.8, 0.6]
])

def rotate(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s], [s, c]])

cov_truth = np.array([np.diag([1, 0.3]) / 10 for i in range(3)])
for c, d in zip(cov_truth, [-1, 1, -1]):
    r = rotate(d * np.pi / 4)
    c[:] = r.T.dot(c).dot(r)

for m, c in zip(mean_truth, cov_truth):
    print(m, c)
    samples = np.random.normal(size=(500, 2))
    samples = samples.dot(c) + m
    plt.scatter(samples[:, 0], samples[:, 1], alpha=0.1)

[ 0.2  0.4] [[ 0.065  0.035]
 [ 0.035  0.065]]
[ 0.5  0.5] [[ 0.065 -0.035]
 [-0.035  0.065]]
[ 0.8  0.6] [[ 0.065  0.035]
 [ 0.035  0.065]]
